# Hikvision NVR 即時影像存取與錄影系統 - Google Colab 版本 (修正版 V2)

本 Notebook 提供了一個完整的解決方案，用於存取 Hikvision DS-7108N-F1/8P NVR 的即時影像，並能依時段進行存檔。這是「出入口即時包裹與人物意圖偵測系統」的第一階段實現，特別針對 Google Colab 環境進行了優化。

## 功能特點

- 透過 RTSP 協議連接到 Hikvision NVR 攝影機
- 支援使用者認證（帳號/密碼）
- 支援多個攝影機通道
- 支援主碼流和子碼流選擇
- 依時段自動分割錄影檔案
- 即時預覽攝影機畫面
- 支援 Google Drive 儲存錄影檔案
- 提供錄影檔案下載功能
- 完整的錯誤處理和日誌記錄

## 1. 安裝必要的套件

首先，我們需要安裝必要的套件。在 Google Colab 中，OpenCV 和 NumPy 已經預先安裝，但我們還需要確保其他相依套件也已安裝。

In [ ]:
# 安裝必要的套件
!pip install opencv-python-headless numpy matplotlib

## 2. Hikvision NVR 攝影機存取模組

以下是用於存取 Hikvision NVR 攝影機的主要模組。這個模組提供了連接到 NVR、取得即時影像、錄影和預覽等功能。

In [ ]:
import cv2
import numpy as np
import os
import time
import datetime
import logging
from threading import Thread
from google.colab import files
from google.colab import drive
import matplotlib.pyplot as plt
from IPython.display import display, clear_output

# Configure logging
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)
logger = logging.getLogger("HikvisionNVR")

class HikvisionNVRCamera:
    """Class to handle Hikvision NVR camera access and recording."""

    def __init__(self, nvr_ip, rtsp_port=554, username=None, password=None,
                 channel=1, stream_type=1, output_dir="recordings", use_drive=False):
        """
        Initialize the Hikvision NVR camera connection.

        Args:
            nvr_ip (str): IP address of the NVR
            rtsp_port (int): RTSP port (default: 554)
            username (str): Username for authentication
            password (str): Password for authentication
            channel (int): Camera channel number (default: 1)
            stream_type (int): Stream type (1 for main stream, 2 for sub stream)
            output_dir (str): Directory to save recordings
            use_drive (bool): Whether to use Google Drive for storage
        """
        self.nvr_ip = nvr_ip
        self.rtsp_port = rtsp_port
        self.username = username
        self.password = password
        self.channel = channel
        self.stream_type = stream_type
        self.use_drive = use_drive

        # Set up output directory
        if use_drive:
            # Mount Google Drive if not already mounted
            if not os.path.exists('/content/drive'):
                print("Mounting Google Drive...")
                drive.mount('/content/drive')

            # Use a directory in Google Drive
            self.output_dir = f"/content/drive/MyDrive/{output_dir}"
        else:
            # Use local Colab directory
            self.output_dir = f"/content/{output_dir}"

        self.cap = None
        self.is_recording = False
        self.current_writer = None
        self.current_output_file = None
        self.preview_thread = None
        self.is_previewing = False
        self.latest_frame = None

        # Create output directory if it doesn't exist
        if not os.path.exists(self.output_dir):
            os.makedirs(self.output_dir)

        # Construct RTSP URL
        self.rtsp_url = self._construct_rtsp_url()

    def _construct_rtsp_url(self):
        """
        Construct the RTSP URL based on the provided parameters.

        Returns:
            str: RTSP URL for the camera stream
        """
        # Format: rtsp://username:password@ip:port/Streaming/channels/channel_number+stream_type
        channel_stream = f"{self.channel}0{self.stream_type}"

        if self.username and self.password:
            return f"rtsp://{self.username}:{self.password}@{self.nvr_ip}:{self.rtsp_port}/Streaming/channels/{channel_stream}"
        else:
            return f"rtsp://{self.nvr_ip}:{self.rtsp_port}/Streaming/channels/{channel_stream}"

    def connect(self):
        """
        Connect to the camera stream.

        Returns:
            bool: True if connection is successful, False otherwise
        """
        logger.info(f"Connecting to camera at {self.rtsp_url}")

        # Create VideoCapture object
        self.cap = cv2.VideoCapture(self.rtsp_url)

        # Check if connection is successful
        if not self.cap.isOpened():
            logger.error("Failed to connect to the camera")
            return False

        logger.info("Successfully connected to the camera")
        return True

    def disconnect(self):
        """Disconnect from the camera stream."""
        if self.cap and self.cap.isOpened():
            self.cap.release()
            logger.info("Disconnected from the camera")

    def start_recording(self, duration_seconds=3600, fps=None):
        """
        Start recording video from the camera.

        Args:
            duration_seconds (int): Duration of each video segment in seconds
            fps (int): Frames per second for recording (None to use camera's FPS)
        """
        if not self.cap or not self.cap.isOpened():
            logger.error("Camera is not connected. Call connect() first.")
            return

        self.is_recording = True

        # Get video properties
        if fps is None:
            fps = int(self.cap.get(cv2.CAP_PROP_FPS))
            if fps <= 0:  # If FPS is not available or invalid
                fps = 30  # Default to 30 FPS

        width = int(self.cap.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(self.cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

        logger.info(f"Video properties: {width}x{height} at {fps} FPS")

        # Start recording thread
        recording_thread = Thread(target=self._recording_loop,
                                 args=(duration_seconds, fps, width, height))
        recording_thread.daemon = True
        recording_thread.start()

        logger.info("Recording started")
        print("Recording started. Videos will be saved to:", self.output_dir)

    def stop_recording(self):
        """Stop recording video."""
        self.is_recording = False
        if self.current_writer:
            self.current_writer.release()
            self.current_writer = None
            logger.info(f"Recording stopped and saved to {self.current_output_file}")
            print(f"Recording stopped and saved to {self.current_output_file}")

    def _recording_loop(self, duration_seconds, fps, width, height):
        """
        Recording loop to capture frames and save to file.

        Args:
            duration_seconds (int): Duration of each video segment in seconds
            fps (int): Frames per second
            width (int): Frame width
            height (int): Frame height
        """
        segment_start_time = time.time()
        frames_in_segment = 0

        # Create initial video writer
        self._create_new_video_writer(fps, width, height)

        while self.is_recording:
            # Read frame from camera
            ret, frame = self.cap.read()

            if not ret:
                logger.warning("Failed to read frame from camera")
                # Try to reconnect
                self.disconnect()
                time.sleep(2)
                if self.connect():
                    continue
                else:
                    break

            # Store latest frame for preview
            self.latest_frame = frame.copy()

            # Write frame to video file
            if self.current_writer:
                self.current_writer.write(frame)
                frames_in_segment += 1

            # Check if it's time to start a new segment
            current_time = time.time()
            if current_time - segment_start_time >= duration_seconds:
                # Close current writer
                if self.current_writer:
                    self.current_writer.release()

                # Create new writer
                self._create_new_video_writer(fps, width, height)

                # Reset segment variables
                segment_start_time = current_time
                frames_in_segment = 0

                logger.info(f"Started new video segment: {self.current_output_file}")
                print(f"Started new video segment: {self.current_output_file}")

    def _create_new_video_writer(self, fps, width, height):
        """
        Create a new video writer for a new segment.

        Args:
            fps (int): Frames per second
            width (int): Frame width
            height (int): Frame height
        """
        # Generate filename with timestamp
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        filename = f"cam{self.channel}_{timestamp}.mp4"
        self.current_output_file = os.path.join(self.output_dir, filename)

        # Create video writer
        fourcc = cv2.VideoWriter_fourcc(*'mp4v')
        self.current_writer = cv2.VideoWriter(
            self.current_output_file, fourcc, fps, (width, height)
        )

        if not self.current_writer.isOpened():
            logger.error(f"Failed to create video writer for {self.current_output_file}")
            self.current_writer = None

    def start_preview(self, interval=1.0):
        """
        Start displaying preview frames in the notebook.
        This is a fixed version that works reliably in Colab.

        Args:
            interval (float): Update interval in seconds
        """
        if not self.cap or not self.cap.isOpened():
            logger.error("Camera is not connected. Call connect() first.")
            return

        self.is_previewing = True

        # Create preview thread
        self.preview_thread = Thread(target=self._preview_loop, args=(interval,))
        self.preview_thread.daemon = True
        self.preview_thread.start()

        logger.info("Preview started")
        print("Preview started. Images will update every", interval, "seconds.")
        print("Note: You will see the preview images appear below.")

    def stop_preview(self):
        """Stop displaying preview frames."""
        self.is_previewing = False
        logger.info("Preview stopped")
        print("Preview stopped.")

    def _preview_loop(self, interval):
        """
        Preview loop to display frames in the notebook.
        This is a fixed version that works reliably in Colab.

        Args:
            interval (float): Update interval in seconds
        """
        while self.is_previewing:
            # Capture a fresh frame directly
            ret, frame = self.cap.read()

            if ret:
                # Convert BGR to RGB for display
                frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

                # Clear previous output and display new frame
                clear_output(wait=True)
                plt.figure(figsize=(10, 8))
                plt.imshow(frame_rgb)
                plt.axis('off')
                plt.title(f"Camera {self.channel} - Live Preview (Updated: {datetime.datetime.now().strftime('%H:%M:%S')})")
                plt.show()
            else:
                logger.warning("Failed to read frame in preview loop")
                time.sleep(1)  # Wait a bit before trying again

            # Wait for the next update
            time.sleep(interval)

    def show_single_frame(self):
        """
        Display a single frame from the camera.
        This improved version ensures a fresh frame is captured each time.

        Returns:
            bool: True if frame was displayed, False otherwise
        """
        if not self.cap or not self.cap.isOpened():
            logger.error("Camera is not connected. Call connect() first.")
            return False

        # Reset the buffer by grabbing a few frames and discarding them
        for _ in range(3):
            self.cap.grab()

        # Read a fresh frame from camera
        ret, frame = self.cap.read()

        if not ret:
            logger.error("Failed to read frame from camera")
            return False

        # Convert BGR to RGB for display
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

        # Display the frame
        plt.figure(figsize=(10, 8))
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.title(f"Camera {self.channel} - Single Frame ({datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')})")
        plt.show()

        return True

    def capture_snapshot(self, filename=None):
        """
        Capture a snapshot from the camera.

        Args:
            filename (str): Optional filename for the snapshot

        Returns:
            str: Path to the saved snapshot file
        """
        if not self.cap or not self.cap.isOpened():
            logger.error("Camera is not connected. Call connect() first.")
            return None

        # Reset the buffer by grabbing a few frames and discarding them
        for _ in range(3):
            self.cap.grab()

        # Read a fresh frame from camera
        ret, frame = self.cap.read()

        if not ret:
            logger.error("Failed to capture snapshot")
            return None

        # Generate filename if not provided
        if filename is None:
            timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"snapshot_cam{self.channel}_{timestamp}.jpg"

        # Save the snapshot
        snapshot_path = os.path.join(self.output_dir, filename)
        cv2.imwrite(snapshot_path, frame)

        logger.info(f"Snapshot saved to {snapshot_path}")
        print(f"Snapshot saved to {snapshot_path}")

        # Display the snapshot
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        plt.figure(figsize=(10, 8))
        plt.imshow(frame_rgb)
        plt.axis('off')
        plt.title("Captured Snapshot")
        plt.show()

        return snapshot_path

    def download_recording(self, filename=None):
        """
        Download a recording file from Colab to local machine.

        Args:
            filename (str): Name of the file to download (if None, downloads the latest recording)
        """
        if filename is None and self.current_output_file:
            filename = os.path.basename(self.current_output_file)

        if filename:
            file_path = os.path.join(self.output_dir, filename)
            if os.path.exists(file_path):
                files.download(file_path)
                print(f"Downloading {file_path}...")
            else:
                print(f"File not found: {file_path}")
        else:
            print("No recording file available for download")

    def list_recordings(self):
        """
        List all recording files in the output directory.

        Returns:
            list: List of recording filenames
        """
        if os.path.exists(self.output_dir):
            recordings = [f for f in os.listdir(self.output_dir) if f.endswith('.mp4')]
            recordings.sort(reverse=True)  # Sort by newest first

            if recordings:
                print(f"Found {len(recordings)} recordings:")
                for i, rec in enumerate(recordings):
                    print(f"{i+1}. {rec}")
            else:
                print("No recordings found")

            return recordings
        else:
            print(f"Output directory not found: {self.output_dir}")
            return []

# Helper functions
def test_connection(rtsp_url, timeout=10):
    """
    Test connection to the Hikvision NVR camera.

    Args:
        rtsp_url (str): RTSP URL for the camera stream
        timeout (int): Connection timeout in seconds

    Returns:
        bool: True if connection is successful, False otherwise
    """
    print(f"Testing connection to: {rtsp_url}")
    print("Attempting to connect (this may take a few seconds)...")

    # Create VideoCapture object
    cap = cv2.VideoCapture(rtsp_url)

    # Wait for connection to establish
    start_time = time.time()
    connected = False

    while time.time() - start_time < timeout:
        if cap.isOpened():
            connected = True
            break
        time.sleep(0.5)

    if not connected:
        print("Failed to connect to the camera within the timeout period.")
        cap.release()
        return False

    # Try to read a frame
    ret, frame = cap.read()

    if not ret:
        print("Connected to camera but failed to read frame.")
        cap.release()
        return False

    # Get video properties
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps = cap.get(cv2.CAP_PROP_FPS)

    print("\nConnection successful!")
    print(f"Video properties: {width}x{height} at {fps} FPS")

    # Display a sample frame
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(10, 8))
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.title("Sample Frame from Camera")
    plt.show()

    # Release the capture
    cap.release()
    return True

def construct_rtsp_url(ip, port, username, password, channel, stream_type):
    """
    Construct the RTSP URL based on the provided parameters.

    Args:
        ip (str): IP address of the NVR
        port (int): RTSP port
        username (str): Username for authentication
        password (str): Password for authentication
        channel (int): Camera channel number
        stream_type (int): Stream type (1 for main stream, 2 for sub stream)

    Returns:
        str: RTSP URL for the camera stream
    """
    # Format: rtsp://username:password@ip:port/Streaming/channels/channel_number+stream_type
    channel_stream = f"{channel}0{stream_type}"

    if username and password:
        return f"rtsp://{username}:{password}@{ip}:{port}/Streaming/channels/{channel_stream}"
    else:
        return f"rtsp://{ip}:{port}/Streaming/channels/{channel_stream}"

## 3. 測試連接到 Hikvision NVR

在開始錄影之前，我們應該先測試與 NVR 的連接。以下是一個測試連接的範例。請將 NVR 的 IP 地址、端口、用戶名和密碼替換為您的實際值。

In [ ]:
# 設定 NVR 參數
NVR_IP = "192.168.1.1"  # 請更改為您的 NVR IP 地址
RTSP_PORT = 554
USERNAME = "username"  # 請更改為您的 NVR 使用者名稱
PASSWORD = "password"  # 請更改為您的 NVR 密碼
CHANNEL = 1  # 攝影機通道號碼
STREAM_TYPE = 1  # 1 為主碼流，2 為子碼流

# 構建 RTSP URL
rtsp_url = construct_rtsp_url(
    NVR_IP, RTSP_PORT, USERNAME, PASSWORD, CHANNEL, STREAM_TYPE
)

# 測試連接
connection_successful = test_connection(rtsp_url, timeout=15)

In [ ]:
def construct_XVR_rtsp_url(ip, port, username, password, channel, stream_type):
    """
    Construct the RTSP URL based on the provided parameters.

    Args:
        ip (str): IP address of the NVR
        port (int): RTSP port
        username (str): Username for authentication
        password (str): Password for authentication
        channel (int): Camera channel number
        stream_type (int): Stream type (1 for main stream, 2 for sub stream)

    Returns:
        str: RTSP URL for the camera stream
    """
    # Format: rtsp://username:password@ip:port/Streaming/channels/channel_number+stream_type
    '''
    rtsp://<用戶名>:<密碼>@<IP地址>:<端口>/cam/realmonitor?channel=1&subtype=0
    channel: 頻道, 1-16; subtype: 碼流類型, 主碼流 0, 輔碼流 1.
    '''
    channel_stream = f"{channel}0{stream_type}"

    if username and password:
        return f"rtsp://{username}:{password}@{ip}:{port}/cam/realmonitor?channel={channel}&subtype={stream_type}"
    else:
        return f"rtsp://{ip}:{port}/cam/realmonitor?channel={channel}&subtype={stream_type}"

In [ ]:
# 設定 環中 XVR 參數
NVR_IP = "192.168.2.1"  # 請更改為您的 NVR IP 地址
RTSP_PORT = 554
USERNAME = "username"  # 請更改為您的 NVR 使用者名稱
PASSWORD = "password"  # 請更改為您的 NVR 密碼
CHANNEL = 1  # 攝影機通道號碼
STREAM_TYPE = 0  # 0 為主碼流，1 為子碼流


# 構建 RTSP URL
rtsp_url = construct_XVR_rtsp_url(
    NVR_IP, RTSP_PORT, USERNAME, PASSWORD, CHANNEL, STREAM_TYPE
)

# 測試連接
connection_successful = test_connection(rtsp_url, timeout=15)

## 4. 使用 HikvisionNVRCamera 類別

如果連接測試成功，我們可以使用 `HikvisionNVRCamera` 類別來存取攝影機並進行錄影。

In [ ]:
# 創建攝影機物件
camera = HikvisionNVRCamera(
    nvr_ip=NVR_IP,
    rtsp_port=RTSP_PORT,
    username=USERNAME,
    password=PASSWORD,
    channel=CHANNEL,
    stream_type=STREAM_TYPE,
    output_dir="hikvision_recordings",  # 錄影檔案儲存目錄
    use_drive=False  # 設為 True 以使用 Google Drive 儲存錄影檔案
)

### 4.1 連接到攝影機

In [ ]:
# 連接到攝影機
if camera.connect():
    print("成功連接到攝影機！")
else:
    print("無法連接到攝影機，請檢查 NVR 設定和網路連接。")

成功連接到攝影機！


### 4.2 顯示單一畫面 (改進版)

我們可以使用 `show_single_frame()` 方法來顯示攝影機的單一畫面。這個改進版本會確保每次都讀取最新的即時畫面，而不是顯示上一次的畫面。

In [ ]:
# 顯示單一畫面
camera.show_single_frame()

### 4.3 開始預覽攝影機畫面 (改進版)

我們可以使用 `start_preview()` 方法來顯示攝影機的即時畫面。這個版本已經改進，可以在 Colab 中正常運作，並且每次都會讀取最新的畫面。

In [ ]:
# 開始預覽攝影機畫面（每 1 秒更新一次）
camera.start_preview(interval=1.0)

Preview started. Images will update every 1.0 seconds.
Note: You will see the preview images appear below.


### 4.4 停止預覽

In [ ]:
# 停止預覽
camera.stop_preview()

Preview stopped.


### 4.5 擷取快照 (改進版)

我們可以使用 `capture_snapshot()` 方法來擷取攝影機的快照。這個改進版本會確保每次都擷取最新的畫面。

In [ ]:
# 擷取快照
snapshot_path = camera.capture_snapshot()

### 4.6 開始錄影

我們可以使用 `start_recording()` 方法來開始錄影。預設情況下，每小時（3600 秒）會建立一個新的影片檔案。

In [ ]:
# 開始錄影，每 10 分鐘（600 秒）建立一個新的影片檔案
camera.start_recording(duration_seconds=600)

### 4.7 停止錄影

In [ ]:
# 停止錄影
camera.stop_recording()

### 4.8 列出錄影檔案

In [ ]:
# 列出錄影檔案
recordings = camera.list_recordings()

### 4.9 下載錄影檔案

我們可以使用 `download_recording()` 方法來下載錄影檔案到本地電腦。

In [ ]:
# 下載最新的錄影檔案
camera.download_recording()

### 4.10 斷開連接

完成所有操作後，我們應該斷開與攝影機的連接。

In [ ]:
# 斷開連接
camera.disconnect()

## 5. 完整使用範例

以下是一個完整的使用範例，展示如何連接到攝影機、預覽畫面、錄影和下載錄影檔案。

In [ ]:
# 設定 NVR 參數
NVR_IP = "192.168.1.100"  # 請更改為您的 NVR IP 地址
RTSP_PORT = 554
USERNAME = "admin"  # 請更改為您的 NVR 使用者名稱
PASSWORD = "password"  # 請更改為您的 NVR 密碼
CHANNEL = 1  # 攝影機通道號碼
STREAM_TYPE = 1  # 1 為主碼流，2 為子碼流
OUTPUT_DIR = "hikvision_recordings"  # 錄影檔案儲存目錄
USE_DRIVE = True  # 是否使用 Google Drive 儲存錄影檔案

try:
    # 創建攝影機物件
    camera = HikvisionNVRCamera(
        nvr_ip=NVR_IP,
        rtsp_port=RTSP_PORT,
        username=USERNAME,
        password=PASSWORD,
        channel=CHANNEL,
        stream_type=STREAM_TYPE,
        output_dir=OUTPUT_DIR,
        use_drive=USE_DRIVE
    )

    # 連接到攝影機
    if camera.connect():
        print("成功連接到攝影機！")

        # 顯示單一畫面
        print("顯示單一畫面...")
        camera.show_single_frame()

        # 開始預覽攝影機畫面
        print("開始預覽攝影機畫面...")
        camera.start_preview(interval=1.0)

        # 等待 5 秒
        time.sleep(5)

        # 停止預覽
        print("停止預覽...")
        camera.stop_preview()

        # 擷取快照
        print("擷取快照...")
        snapshot_path = camera.capture_snapshot()

        # 開始錄影
        print("開始錄影...")
        camera.start_recording(duration_seconds=60)  # 每 60 秒建立一個新的影片檔案

        # 錄影 10 秒
        time.sleep(10)

        # 停止錄影
        print("停止錄影...")
        camera.stop_recording()

        # 列出錄影檔案
        print("列出錄影檔案...")
        recordings = camera.list_recordings()

        # 下載最新的錄影檔案
        if recordings:
            print("下載最新的錄影檔案...")
            camera.download_recording(recordings[0])
    else:
        print("無法連接到攝影機，請檢查 NVR 設定和網路連接。")
except Exception as e:
    print(f"發生錯誤: {e}")
finally:
    # 斷開連接
    if 'camera' in locals() and camera.cap is not None:
        print("斷開連接...")
        camera.disconnect()

## 6. 預覽功能說明與故障排除

在 Google Colab 環境中，預覽功能可能會遇到一些限制。以下是一些說明和故障排除方法：

### 預覽方式

本程式提供兩種預覽方式：

1. **連續預覽** (`start_preview()`)：持續更新顯示攝影機畫面，適合監控即時狀況。
2. **單一畫面** (`show_single_frame()`)：顯示攝影機的單一畫面，適合快速檢查攝影機狀態。

### 改進的畫面更新機制

在最新版本中，我們對畫面更新機制進行了以下改進：

1. **緩衝區清除**：在 `show_single_frame()` 和 `capture_snapshot()` 函式中，我們會先清除緩衝區，確保每次都能獲取最新的畫面。

2. **直接讀取畫面**：在 `_preview_loop()` 中，我們直接從攝影機讀取畫面，而不是使用儲存的 `latest_frame`，確保預覽顯示的是最新畫面。

3. **錯誤處理**：增強了錯誤處理機制，當無法讀取畫面時會提供適當的警告訊息。

### 常見問題與解決方法

1. **預覽畫面沒有顯示**：
   - 確保攝影機已成功連接（`connect()` 返回 `True`）
   - 嘗試使用 `show_single_frame()` 來檢查是否能獲取畫面
   - 增加更新間隔時間（例如：`interval=2.0`）
   - 重新執行儲存格

2. **預覽畫面更新緩慢**：
   - 增加更新間隔時間（例如：`interval=2.0` 或更高）
   - 使用子碼流（`stream_type=2`）以降低解析度
   - 檢查網路連接速度

3. **預覽畫面凍結**：
   - 停止預覽（`stop_preview()`）
   - 重新連接攝影機（`disconnect()` 然後 `connect()`）
   - 重新啟動預覽（`start_preview()`）

4. **記憶體不足錯誤**：
   - 增加更新間隔時間
   - 使用子碼流（`stream_type=2`）
   - 使用 `show_single_frame()` 代替連續預覽
   - 重新啟動 Colab 執行階段

### 預覽最佳實踐

1. 先使用 `show_single_frame()` 確認攝影機連接正常
2. 使用 `start_preview(interval=1.0)` 開始連續預覽
3. 完成預覽後，使用 `stop_preview()` 停止預覽
4. 如果需要查看最新畫面，可以多次使用 `show_single_frame()` 來查看攝影機畫面

## 7. Google Colab 特定考量事項

在 Google Colab 中使用此程式碼時，有一些特定的考量事項需要注意：

1. **網路連接**：Colab 運行在 Google 的伺服器上，無法直接存取您的本地網路。您需要確保 NVR 可以從公共網路存取，或者使用 VPN 或其他方式將 Colab 連接到您的本地網路。

2. **Google Drive 整合**：為了長期儲存錄影檔案，建議使用 Google Drive。在上面的程式碼中，將 `use_drive` 參數設為 `True` 可以將錄影檔案儲存到您的 Google Drive。

3. **Colab 執行時間限制**：Google Colab 有執行時間限制，免費版本通常在閒置一段時間後會自動斷開連接。如果您需要長時間錄影，可能需要考慮使用 Colab Pro 或在本地環境中運行程式碼。

4. **記憶體限制**：處理高解析度影像和影片可能會消耗大量記憶體。如果遇到記憶體不足的問題，可以考慮使用子碼流（stream_type=2）來降低解析度，或者減少預覽更新頻率。

5. **下載檔案**：Colab 提供了方便的檔案下載功能，但大型檔案的下載可能會很慢。如果您需要下載大量錄影檔案，建議將它們儲存到 Google Drive，然後從 Google Drive 下載。

## 8. 故障排除

如果您在使用此程式碼時遇到問題，以下是一些常見問題的解決方法：

1. **無法連接到攝影機**：
   - 確保 NVR 的 IP 地址、端口、用戶名和密碼正確
   - 確保 NVR 可以從 Colab 環境存取（可能需要公共 IP 或 VPN）
   - 確保 NVR 已啟用 RTSP 服務
   - 嘗試使用 VLC 或其他 RTSP 播放器測試 RTSP URL

2. **連接成功但無法讀取影像**：
   - 確保攝影機通道號碼正確
   - 嘗試使用子碼流（stream_type=2）
   - 檢查 NVR 的 RTSP 串流設定

3. **錄影檔案無法播放**：
   - 確保錄影時間足夠長（至少幾秒鐘）
   - 嘗試使用不同的影片播放器（如 VLC）
   - 檢查錄影檔案的大小，如果太小可能表示沒有正確錄製

4. **Google Drive 掛載失敗**：
   - 確保您已登入 Google 帳戶
   - 嘗試重新執行 `drive.mount('/content/drive')` 命令
   - 如果仍然失敗，嘗試使用本地儲存（將 `use_drive` 設為 `False`）

5. **記憶體不足錯誤**：
   - 減少預覽更新頻率
   - 使用子碼流（stream_type=2）降低解析度
   - 重新啟動 Colab 執行階段並釋放記憶體

## 9. 更新日誌

### 版本 V2 (2025-04-27)
- 修正了 `show_single_frame()` 函式，確保每次都讀取最新的即時畫面
- 改進了 `_preview_loop()` 方法，直接從攝影機讀取畫面而不是使用儲存的 latest_frame
- 改進了 `capture_snapshot()` 函式，確保每次都擷取最新的畫面
- 增加了緩衝區清除機制，在讀取畫面前先清除緩衝區
- 增強了錯誤處理機制

### 版本 V1 (2025-04-25)
- 初始版本
- 實現了基本的 NVR 攝影機存取功能
- 實現了錄影和預覽功能
- 支援 Google Drive 整合
- 提供了完整的使用範例和說明